# Snowflake Setup - データベース、スキーマ、ウェアハウスの作成

このNotebookは、Tasty Bytesプロジェクトの初期セットアップを行います。

In [ ]:
%%sql
USE ROLE sysadmin;

## データベースとスキーマの作成

In [ ]:
%%sql
CREATE OR REPLACE DATABASE tb_101;
CREATE OR REPLACE SCHEMA tb_101.raw_pos;
CREATE OR REPLACE SCHEMA tb_101.raw_customer;
CREATE OR REPLACE SCHEMA tb_101.harmonized;
CREATE OR REPLACE SCHEMA tb_101.analytics;
CREATE OR REPLACE SCHEMA tb_101.governance;
CREATE OR REPLACE SCHEMA tb_101.raw_support;
CREATE OR REPLACE SCHEMA tb_101.semantic_layer
COMMENT = '分析用途に最適化された、ビジネスに優しいセマンティックレイヤー用スキーマ';

## ウェアハウスの作成

In [ ]:
%%sql
CREATE OR REPLACE WAREHOUSE tb_de_wh
    WAREHOUSE_SIZE = 'large'
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'Tasty Bytes用データエンジニアリングウェアハウス';

CREATE OR REPLACE WAREHOUSE tb_dev_wh
    WAREHOUSE_SIZE = 'xsmall'
    WAREHOUSE_TYPE = 'standard'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'Tasty Bytes用開発者ウェアハウス';

CREATE OR REPLACE WAREHOUSE tb_analyst_wh
    WAREHOUSE_TYPE = 'standard'
    WAREHOUSE_SIZE = 'large'
    MIN_CLUSTER_COUNT = 1
    MAX_CLUSTER_COUNT = 2
    SCALING_POLICY = 'standard'
    AUTO_SUSPEND = 60
    INITIALLY_SUSPENDED = true,
    AUTO_RESUME = true
COMMENT = 'TastyBytesアナリストウェアハウス';

CREATE OR REPLACE WAREHOUSE tb_cortex_wh
    WAREHOUSE_SIZE = 'LARGE'
    WAREHOUSE_TYPE = 'STANDARD'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = TRUE
COMMENT = 'Cortex Analystとその他の分析ツール用の専用大型ウェアハウス';

## ロールの作成

In [ ]:
%%sql
USE ROLE securityadmin;

CREATE ROLE IF NOT EXISTS tb_admin COMMENT = 'Tasty Bytes管理者';
CREATE ROLE IF NOT EXISTS tb_data_engineer COMMENT = 'Tasty Bytesデータエンジニア';
CREATE ROLE IF NOT EXISTS tb_dev COMMENT = 'Tasty Bytes開発者';
CREATE ROLE IF NOT EXISTS tb_analyst COMMENT = 'Tasty Bytesアナリスト';

GRANT ROLE tb_admin TO ROLE sysadmin;
GRANT ROLE tb_data_engineer TO ROLE tb_admin;
GRANT ROLE tb_dev TO ROLE tb_data_engineer;
GRANT ROLE tb_analyst TO ROLE tb_data_engineer;

## 権限付与

In [ ]:
%%sql
USE ROLE accountadmin;
GRANT IMPORTED PRIVILEGES ON DATABASE snowflake TO ROLE tb_data_engineer;
GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE tb_admin;

USE ROLE securityadmin;
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_admin;
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_data_engineer;
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_dev;

GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_admin;
GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_data_engineer;
GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_dev;

## ファイル形式とステージの作成

In [ ]:
%%sql
USE ROLE sysadmin;
USE WAREHOUSE tb_de_wh;

CREATE OR REPLACE FILE FORMAT tb_101.public.csv_ff type = 'csv';

CREATE OR REPLACE STAGE tb_101.public.s3load
COMMENT = 'クイックスタートS3ステージ接続'
url = 's3://sfquickstarts/frostbyte_tastybytes/'
file_format = tb_101.public.csv_ff;

CREATE OR REPLACE STAGE tb_101.public.truck_reviews_s3load
COMMENT = 'トラックレビューステージ'
url = 's3://sfquickstarts/tastybytes-voc/'
file_format = tb_101.public.csv_ff;

## データのロード

残りのテーブル作成とデータロードの詳細は、元のSQLスクリプトを参照してください。